In [4]:
import bnNB
import random
import math

I utilize the code I wrote for homework [Naive Bayesian Implementation](https://github.com/peleonard/CMPS-4720-6720-Sp18/blob/master/NaiveBayesImplem/NaiveBayesSPECTScratch.ipynb). 

The main body of this code is a semi-supervised Bernoulli Naive Bayes. Please find the [reference](https://www.cs.cmu.edu/~tom/pubs/NigamEtAl-bookChapter.pdf). It's on page 37. And for the stopping criterion, I observe the change of loglikelihood defined in formula (3.8). If it gets stable, error for two consecutive loglikelihood is smalled than 1e-10, I shutdown the iteration. 

In [5]:
Strain = bnNB.parseTxtSPECT("SPECTtrain.txt")
Stest = bnNB.parseTxtSPECT("SPECTtest.txt")

low = 12
up = 70

StrainLabeled = []
for i in range(len(Strain)):
    if i < low or i > up:
        StrainLabeled.append(Strain[i])
 
StrainUnlabeled = Strain[low:(up+1)]

semiNBFit0 = bnNB.meanByClass(StrainLabeled)

noise = 1e-2

def llLabeled(fit, datasetLabeled):
    ll = []
    for i in range(len(datasetLabeled)):
        ll0 = fit[1][datasetLabeled[i][0]]
        for j in range(len(datasetLabeled[0]) - 1):
            ll0 *= bnNB.Bernoulli(datasetLabeled[i][j + 1], 
                             fit[0][datasetLabeled[i][j + 1]][j])
        ll.append(math.log(ll0 + 1e-2))
    return(sum(ll))
 
llLabeled(semiNBFit0, StrainLabeled)


def llUnlabeled(fit, probs, datasetUnlabeled):
    ll = []
    for i in range(len(datasetUnlabeled)):
        ll0_0 = probs[i][0]
        ll1_0 = probs[i][0]
        for j in range(len(datasetUnlabeled[0]) - 1):
            ll0_0 *= bnNB.Bernoulli(datasetUnlabeled[i][j + 1],
                               fit[0][0][j])
            ll1_0 *= bnNB.Bernoulli(datasetUnlabeled[i][j + 1],
                               fit[0][1][j])
        
        ll.append(math.log(ll0_0 + ll1_0 + 1e-2))
        
    return(sum(ll))



T = 100
LL = []
error = 1e-10
for t in range(T):
    
    unlabeledProbs = []
    for i in range(len(StrainUnlabeled)):
        unlabeledProb = bnNB.testProb(semiNBFit0, StrainUnlabeled[i][1:])    
        unlabeledProbs.append(unlabeledProb)
    
    unlabeledPred = bnNB.bernoulliNBPredTestset(semiNBFit0, StrainUnlabeled)
     
    for i in range(len(StrainUnlabeled)):
        StrainUnlabeled[i][0] = unlabeledPred[i]
    
    semiNBFit = bnNB.meanByClass(StrainLabeled + StrainUnlabeled)
    
    loglikeLabeled = llLabeled(semiNBFit, StrainLabeled) 
    loglikeUnlabeled = llUnlabeled(semiNBFit, unlabeledProbs, StrainUnlabeled)
    LL.append(loglikeLabeled + loglikeUnlabeled)
    
    if (t > 0 and ((LL[t] - LL[t-1]) < error)):
        break
    
    semiNBFit0 = semiNBFit


One can see the decrease of likelihood.

In [6]:
print("The loglikelihood during the iteration are \n [%s]" %(LL))

The loglikelihood during the iteration are 
 [[-363.8978690517987, -359.24471406575634, -358.84038006445564, -358.8037555797348, -358.8037555797348]]


If we only use the labeled data. 

In [7]:
predTest0, accu0 = bnNB.bernoulliNBClassifier(StrainLabeled, Stest)
print("The acuracy of only used labed data is %.3f" %(accu0))

The acuracy of only used labed data is 0.818


Define the main function and run it to obtain the accuracy. We can the model is not improved.

In [8]:
def semiEM_NB(fit, testset):
    predTest = bnNB.bernoulliNBPredTestset(fit, testset)
    accu = bnNB.accuracyNB(predTest, testset)
    return accu, predTest

accu, predTest = semiEM_NB(semiNBFit, Stest)
print("""The predictions of test dataset and accuracy for a semi-supervised 
      Bernoulli Naive Bayes on SPECT dastet is \n [%s] \n and %.3f""" %(predTest, accu))


The predictions of test dataset and accuracy for a semi-supervised 
      Bernoulli Naive Bayes on SPECT dastet is 
 [[1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0]] 
 and 0.770
